In [21]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

# 基本示例

In [139]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("讲一个关于{topic}的笑话")
llm = ChatOpenAI(model="gpt-3.5-turbo-1106")
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

In [107]:
[name for name in model.__fields__]
model.__fields__

{'name': ModelField(name='name', type=Optional[str], required=False, default=None),
 'cache': ModelField(name='cache', type=Optional[bool], required=False, default=None),
 'verbose': ModelField(name='verbose', type=bool, required=False, default_factory='<function _get_verbosity>'),
 'callbacks': ModelField(name='callbacks', type=Union[List[langchain_core.callbacks.base.BaseCallbackHandler], BaseCallbackManager, NoneType], required=False, default=None),
 'callback_manager': ModelField(name='callback_manager', type=Optional[BaseCallbackManager], required=False, default=None),
 'tags': ModelField(name='tags', type=Optional[List[str]], required=False, default=None),
 'metadata': ModelField(name='metadata', type=Optional[Mapping[str, Any]], required=False, default=None),
 'client': ModelField(name='client', type=Optional[Any], required=False, default=None),
 'async_client': ModelField(name='async_client', type=Optional[Any], required=False, default=None),
 'model_name': ModelField(name='mod

In [108]:
model.__annotations__

{'client': 'Any',
 'async_client': 'Any',
 'model_name': 'str',
 'temperature': 'float',
 'model_kwargs': 'Dict[str, Any]',
 'openai_api_key': 'Optional[str]',
 'openai_api_base': 'Optional[str]',
 'openai_organization': 'Optional[str]',
 'openai_proxy': 'Optional[str]',
 'request_timeout': 'Union[float, Tuple[float, float], Any, None]',
 'max_retries': 'int',
 'streaming': 'bool',
 'n': 'int',
 'max_tokens': 'Optional[int]',
 'tiktoken_model_name': 'Optional[str]',
 'default_headers': 'Union[Mapping[str, str], None]',
 'default_query': 'Union[Mapping[str, object], None]',
 'http_client': 'Union[Any, None]'}

In [119]:
from langchain_core.runnables.config import RunnableConfig

In [137]:
RunnableConfig.__annotations__

{'tags': ForwardRef('List[str]'),
 'metadata': ForwardRef('Dict[str, Any]'),
 'callbacks': ForwardRef('Callbacks'),
 'run_name': ForwardRef('str'),
 'max_concurrency': ForwardRef('Optional[int]'),
 'recursion_limit': ForwardRef('int'),
 'configurable': ForwardRef('Dict[str, Any]')}

In [138]:
model._lc_kwargs

{}

# RunnableConfig

```python
class RunnableConfig(TypedDict, total=False):
    """Configuration for a Runnable."""

    tags: List[str]
    """
    Tags for this call and any sub-calls (eg. a Chain calling an LLM).
    You can use these to filter calls.
    """

    metadata: Dict[str, Any]
    """
    Metadata for this call and any sub-calls (eg. a Chain calling an LLM).
    Keys should be strings, values should be JSON-serializable.
    """

    callbacks: Callbacks
    """
    Callbacks for this call and any sub-calls (eg. a Chain calling an LLM).
    Tags are passed to all callbacks, metadata is passed to handle*Start callbacks.
    """

    run_name: str
    """
    Name for the tracer run for this call. Defaults to the name of the class.
    """

    max_concurrency: Optional[int]
    """
    Maximum number of parallel calls to make. If not provided, defaults to 
    ThreadPoolExecutor's default.
    """

    recursion_limit: int
    """
    Maximum number of times a call can recurse. If not provided, defaults to 25.
    """

    configurable: Dict[str, Any]
    """
    Runtime values for attributes previously made configurable on this Runnable,
    or sub-Runnables, through .configurable_fields() or .configurable_alternatives().
    Check .output_schema() for a description of the attributes that have been made 
    configurable.
    """
```

# ConfigurableField

```python
class ConfigurableField(NamedTuple):
    """Field that can be configured by the user."""

    id: str

    name: Optional[str] = None
    description: Optional[str] = None
    annotation: Optional[Any] = None
    is_shared: bool = False

    def __hash__(self) -> int:
        return hash((self.id, self.annotation))
```

In [69]:
from langchain_core.runnables import ConfigurableField

abc = ConfigurableField(
    id="run_name",
    is_shared=True
)
abc

ConfigurableField(id='run_name', name=None, description=None, annotation=None, is_shared=True)

In [71]:
prompt.config_schema(include={"run_name":"ABC"})

pydantic.main.ChatPromptTemplateConfig

# Runnable的可配置类

**config_schema 方法** 使用 `include` 参数和 `config_specs` 方法的返回值来创建一个 pydantic 模型，这个模型可以用来验证这个 Runnable 对象的配置。

`config_specs` 方法应当返回一个 ConfigurableFieldSpec 对象的列表，每个 ConfigurableFieldSpec 对象代表一个可配置的字段。每个 ConfigurableFieldSpec 对象应该有一个 id 属性（字段的名称），一个 annotation 属性（字段的类型），以及一个 default 属性（字段的默认值）。

<div class="alert-info" style="padding:5px">
如果想在你定制的 Runnable 子类中添加可配置的字段，你可以重载<b>config_specs</b>方法并返回一个包含你的字段的 ConfigurableFieldSpec 对象的列表。
</div>

`include 参数`允许你选择 RunnableConfig 中已经定义的字段，这些字段将被包含在新创建的 pydantic 模型中。

所以，`include` 和 `config_specs` 允许你从两个方向来定制你的配置模型：include 允许你选择已经存在的字段，而 config_specs 允许你添加新的字段。

In [55]:
prompt.configurable_fields()

RunnableConfigurableFields(default=ChatPromptTemplate(input_variables=['topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='讲一个关于{topic}的笑话'))]), fields={})

In [54]:
llm.configurable_fields()

RunnableConfigurableFields(default=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1134bfd30>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x113369840>, model_name='gpt-3.5-turbo-1106', openai_api_key='sk-SctYLVDdKoqPLXoKlLEMjLqHG2BhuOtQgjmMWvrTDqlKJeqz', openai_proxy=''), fields={})